Training pipeline can be found here: https://www.kaggle.com/alibaba19/jigsaw-training-ulmfit-w-fastai

In [ ]:
from fastai.text.all import *


In [ ]:
input_path = Path('../input/jigsaw-toxic-severity-rating')
sub_df = pd.read_csv(input_path/'sample_submission.csv')
test_df = pd.read_csv(input_path/'comments_to_score.csv')
df = pd.read_csv(input_path/'comments_to_score.csv')
df['total_tox'] = 5
df.shape

In [ ]:
vocab = pd.read_csv('../input/initial-files/lm_vocab.csv')
lm_vocab = vocab['0'].values

In [ ]:
! mkdir ../working/models


In [ ]:
!cp ../input/initial-files/initial_model.pth ../working/models

In [ ]:
path = Path('../input/initial-files/')

In [ ]:
path.ls()

In [ ]:
toxic_db = DataBlock(blocks=(TextBlock.from_df('text', vocab=lm_vocab), RegressionBlock),
                     get_x = ColReader('text'),
                     get_y = ColReader('total_tox'),
                     splitter = RandomSplitter())


In [ ]:
dls = toxic_db.dataloaders(df[:100], bs=64)
learner = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy, pretrained=False)

In [ ]:
learner.load('initial_model');

In [ ]:
learner.model

In [ ]:
test_dl = dls.test_dl(test_df['text'])
len(test_dl)

In [ ]:
preds = learner.get_preds(dl=test_dl) 
len(preds), preds[0].shape

In [ ]:
sub_df.score = preds[0]
sub_df.to_csv('submission.csv', index=False)